<a href="https://colab.research.google.com/github/JackBenadon/DS440-Project/blob/main/DS440_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import calendar
import datetime
from datetime import datetime, timedelta, date
import warnings
import os
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso


#settings
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 1200)
pd.set_option('display.width', 1200)
warnings.filterwarnings("ignore")
%matplotlib inline

#variables
colour=['maroon','r','g','darkgreen','c','teal','b','navy','indigo','m','deeppink','orange','sienna','yellow','khaki','olive','tan','black','grey','brown']

print("Packages installed")

Packages installed


In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
# import all files in Data folder and read into dataframes
dataframes=[]
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Senior year/Capstone/data/archive'):
    for filename in filenames:
        file=filename.split('.')
        file=((file[0]+"_df"))
        if file !="_df":
            filepath=os.path.join(dirname,filename)
            df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")
            exec(f'{file} = df.copy()')
            print(file, df.shape)
            dataframes.append(df)
print('Data imported')

appearances_df (1540638, 13)
club_games_df (133586, 11)
competitions_df (43, 10)
clubs_df (426, 17)
game_events_df (691486, 10)
game_lineups_df (2145222, 9)
player_valuations_df (465282, 5)
games_df (66793, 23)
players_df (30495, 23)
Data imported


In [70]:
path = "/content/drive/MyDrive/Senior year/Capstone/data/DS440 Master Table.csv"
master_table = pd.read_csv(path, encoding='iso-8859-1')
master_table

,Name,Nationality,Position,Team,Age,YOB,MatchedPlayed,Starts,MinutesPlayed,90s,Goals,Assists,GA,NonPenGoals,PensScored,PensAttempted,YellowCards,RedCards,xG,npxG,xAG,npxG + xAG,ProgressiveCarries,ProgressivePasses,ProgressivePassesReceived,G/90,A/90,GA/90,NonPenGoals/90,NonPenGA/90,xG/90,xAG/90,xA+xG /90,npxG/90,npXG + xAG /90,Matches,Season,Shots,ShotsOnTarget,SoT%,S/90,SoT/90,G/Shot,G/SoT,AvgShotDist,FKShots,npxG/Shot,Goals - xG,npG - npxG,PassesCompleted,PassesAttempted,PercentPassesCompleted,TotalPassingDistance,ProgressedPassingDistance,ShortPassesCompleted,ShortPassesAttempted,ShortPassesPercentCompleted,MedPassesCompleted,MedPassesAttempted,MedPassesPercentCompleted,LongPassesCompleted,LongPassesAttempted,LongPassesPercentCompleted,xA,A - xAG,KeyPasses,1/3,PPen,CPen,Progressive Passes,Tackles,TacklesWon,TklsDefThird,TklsMidThird,TklsAttThird,DribblersTackled,DribblesChallenged,DrblTkl%,ChallengesLost,Blocks,BlockedShots,BlockedPasses,Interceptions,Tackles + Interceptions,Clearances,DefensiveErrors
0,Max Aarons,eng ENG,DF,Bournemouth,24-070,2000,14,12,"1,085",12.1,0,1,1,0,0,0,1,0,0.0,0.0,0.8,0.8,19.0,40.0,22.0,0.00,0.08,0.08,0.00,0.08,0.00,0.07,0.07,0.00,0.07,Matches,2023/2024,1,0,0.0,0.08,0.00,0.00,NaN,23.9,0.0,0.02,0.0,0.0,379.0,501.0,75.6,6256.0,2424.0,186.0,213.0,87.3,156.0,202.0,77.2,31.0,56.0,55.4,0.9,0.2,7.0,22.0,12.0,2.0,40.0,28.0,18,20.0,6.0,2.0,19.0,30.0,63.3,11.0,9.0,5.0,4.0,6,34.0,23.0,0.0
1,Bénie Adama Traore,ci CIV,"FW,MF",Sheffield Utd,21-105,2002,8,3,387,4.3,0,0,0,0,0,0,0,0,0.3,0.3,0.5,0.8,7.0,9.0,14.0,0.00,0.00,0.00,0.00,0.00,0.06,0.13,0.19,0.06,0.19,Matches,2023/2024,1,1,100.0,0.23,0.23,0.00,0.00,15.3,0.0,0.27,-0.3,-0.3,55.0,71.0,77.5,775.0,185.0,34.0,38.0,89.5,19.0,23.0,82.6,1.0,1.0,100.0,0.5,-0.5,4.0,2.0,7.0,1.0,9.0,4.0,2,1.0,2.0,1.0,0.0,4.0,0.0,4.0,4.0,1.0,3.0,1,5.0,1.0,0.0
2,Tyler Adams,us USA,MF,Bournemouth,25-029,1999,1,0,20,0.2,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,Matches,2023/2024,0,0,NaN,0.00,0.00,NaN,NaN,NaN,0.0,NaN,0.0,0.0,11.0,14.0,78.6,152.0,34.0,8.0,8.0,100.0,3.0,5.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,Tosin Adarabioyo,eng ENG,DF,Fulham,26-172,1997,14,12,"1,083",12.0,1,0,1,1,0,0,1,0,0.6,0.6,0.1,0.6,5.0,38.0,2.0,0.08,0.00,0.08,0.08,0.08,0.05,0.01,0.05,0.05,0.05,Matches,2023/2024,10,3,30.0,0.83,0.25,0.10,0.33,15.5,0.0,0.06,0.4,0.4,703.0,830.0,84.7,14444.0,5328.0,189.0,211.0,89.6,432.0,479.0,90.2,77.0,117.0,65.8,0.3,-0.1,1.0,28.0,2.0,0.0,38.0,16.0,10,11.0,5.0,0.0,10.0,14.0,71.4,4.0,11.0,8.0,3.0,18,34.0,60.0,0.0
4,Elijah Adebayo,eng ENG,FW,Luton Town,26-067,1998,23,13,"1,162",12.9,9,0,9,9,0,0,1,0,5.9,5.9,0.7,6.6,15.0,19.0,90.0,0.70,0.00,0.70,0.70,0.70,0.44,0.05,0.49,0.44,0.49,Matches,2023/2024,28,12,42.9,2.17,0.93,0.32,0.75,9.5,0.0,0.20,3.4,3.4,128.0,177.0,72.3,1627.0,336.0,84.0,105.0,80.0,35.0,52.0,67.3,3.0,6.0,50.0,0.4,-0.7,14.0,7.0,3.0,0.0,19.0,1.0,0,1.0,0.0,0.0,0.0,10.0,0.0,10.0,8.0,1.0,7.0,5,6.0,23.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,Wilfried Zaha,ci CIV,"FW,MF",Crystal Palace,26,1992,38,37,"3,280",36.4,4,3,7,4,0,0,5,0,5.2,5.2,4.0,9.1,210.0,146.0,354.0,0.11,0.08,0.19,0.11,0.19,0.14,0.11,0.25,0.14,0.25,Matches,2019/2020,61,17,27.9,1.67,0.47,0.07,0.24,16.2,0.0,0.08,-1.2,-1.2,928.0,1188.0,78.1,13537.0,3574.0,547.0,648.0,84.4,277.0,335.0,82.7,63.0,87.0,72.4,4.7,-1.0,36.0,62.0,70.0,12.0,146.0,43.0,19,18.0,12.0,13.0,12.0,49.0,24.5,37.0,22.0,2.0,20.0,18,61.0,4.0,0.0
2721,Christoph Zimmermann,de GER,DF,Norwich City,26,1993,17,16,"1,430",15.9,0,1,1,0,0,0,3,0,0.4,0.4,0.1,0.5,8.0,33.0,0.0,0.00,0.06,0.06,0.00,0.06,0.02,0.01,0.03,0.02,0.03,Matches,2019/2020,6,1,16

In [71]:
#We mainly want the value of the players, their name, and the date it was updated
merged_table = pd.merge(player_valuations_df, players_df[['player_id', 'name']], on='player_id')
merged_table

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,name
0,405973,2000-01-20,150000,3057,BE1,Fadel Gobitaka
1,405973,2016-02-07,250000,3057,BE1,Fadel Gobitaka
2,405973,2016-07-15,250000,3057,BE1,Fadel Gobitaka
3,405973,2017-01-01,150000,3057,BE1,Fadel Gobitaka
4,405973,2018-09-11,75000,3057,BE1,Fadel Gobitaka
...,...,...,...,...,...,...
465277,993081,2024-02-09,50000,924,TR1,Murat Eser
465278,988247,2024-02-10,50000,6890,TR1,Hamza Güreler
465279,720799,2024-02-12,1000000,610,NL1,Julian Rijkhoff
465280,1146029,2024-02-12,1000000,472,ES1,Juanma Herzog


In [72]:
from datetime import datetime

#this function will convert the date from when the price was updated into the season in which it was updated
def adjust_year(date_string):
    # Parse the date string into a datetime object
    date_obj = datetime.strptime(date_string, '%Y-%m-%d')

    # Define the cutoff date
    cutoff_date = datetime(date_obj.year, 6, 15)

    # Compare the date with the cutoff date
    if date_obj < cutoff_date:
        # If the date is before the cutoff date, use the previous year
        return f"{date_obj.year - 1}/{date_obj.year}"
    else:
        # If the date is on or after the cutoff date, use the current year and the next year
        return f"{date_obj.year}/{date_obj.year + 1}"

# Example usage
date_string = "2012-05-31"
adjusted_year = adjust_year(date_string)
print(adjusted_year)  # Output: 2011/2012

2011/2012


In [73]:
#Applying the function to the table
merged_table['Season'] = merged_table['date'].apply(adjust_year)
merged_table

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,name,Season
0,405973,2000-01-20,150000,3057,BE1,Fadel Gobitaka,1999/2000
1,405973,2016-02-07,250000,3057,BE1,Fadel Gobitaka,2015/2016
2,405973,2016-07-15,250000,3057,BE1,Fadel Gobitaka,2016/2017
3,405973,2017-01-01,150000,3057,BE1,Fadel Gobitaka,2016/2017
4,405973,2018-09-11,75000,3057,BE1,Fadel Gobitaka,2018/2019
...,...,...,...,...,...,...,...
465277,993081,2024-02-09,50000,924,TR1,Murat Eser,2023/2024
465278,988247,2024-02-10,50000,6890,TR1,Hamza Güreler,2023/2024
465279,720799,2024-02-12,1000000,610,NL1,Julian Rijkhoff,2023/2024
465280,1146029,2024-02-12,1000000,472,ES1,Juanma Herzog,2023/2024


In [74]:
#Renaming the columns so they match the other table in order to merge
print(merged_table.columns)
merged_table.columns = merged_table.columns.str.strip()
merged_table.rename(columns={'name': 'Name'}, inplace=True)
merged_table

Index(['player_id', 'date', 'market_value_in_eur', 'current_club_id', 'player_club_domestic_competition_id', 'name', 'Season'], dtype='object')


,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,Name,Season
0,405973,2000-01-20,150000,3057,BE1,Fadel Gobitaka,1999/2000
1,405973,2016-02-07,250000,3057,BE1,Fadel Gobitaka,2015/2016
2,405973,2016-07-15,250000,3057,BE1,Fadel Gobitaka,2016/2017
3,405973,2017-01-01,150000,3057,BE1,Fadel Gobitaka,2016/2017
4,405973,2018-09-11,75000,3057,BE1,Fadel Gobitaka,2018/2019
...,...,...,...,...,...,...,...
465277,993081,2024-02-09,50000,924,TR1,Murat Eser,2023/2024
465278,988247,2024-02-10,50000,6890,TR1,Hamza Güreler,2023/2024
465279,720799,2024-02-12,1000000,610,NL1,Julian Rijkhoff,2023/2024
465280,1146029,2024-02-12,1000000,472,ES1,Juanma Herzog,2023/2024


In [75]:
#making the merge, only keeping the entries that match player name AND year
total_table_df = pd.merge(merged_table, master_table, on=['Name', 'Season'], how='inner')

In [76]:
total_table_df

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,Name,Season,Nationality,Position,Team,Age,YOB,MatchedPlayed,Starts,MinutesPlayed,90s,Goals,Assists,GA,NonPenGoals,PensScored,PensAttempted,YellowCards,RedCards,xG,npxG,xAG,npxG + xAG,ProgressiveCarries,ProgressivePasses,ProgressivePassesReceived,G/90,A/90,GA/90,NonPenGoals/90,NonPenGA/90,xG/90,xAG/90,xA+xG /90,npxG/90,npXG + xAG /90,Matches,Shots,ShotsOnTarget,SoT%,S/90,SoT/90,G/Shot,G/SoT,AvgShotDist,FKShots,npxG/Shot,Goals - xG,npG - npxG,PassesCompleted,PassesAttempted,PercentPassesCompleted,TotalPassingDistance,ProgressedPassingDistance,ShortPassesCompleted,ShortPassesAttempted,ShortPassesPercentCompleted,MedPassesCompleted,MedPassesAttempted,MedPassesPercentCompleted,LongPassesCompleted,LongPassesAttempted,LongPassesPercentCompleted,xA,A - xAG,KeyPasses,1/3,PPen,CPen,Progressive Passes,Tackles,TacklesWon,TklsDefThird,TklsMidThird,TklsAttThird,DribblersTackled,DribblesChallenged,DrblTkl%,ChallengesLost,Blocks,BlockedShots,BlockedPasses,Interceptions,Tackles + Interceptions,Clearances,DefensiveErrors
0,3333,2019-12-10,12000000,1237,GB1,James Milner,2019/2020,eng ENG,"MF,DF",Liverpool,33,1986,22,9,937,10.4,2,2,4,0,2,2,4,0,2.2,0.6,0.7,1.3,18.0,71.0,42.0,0.19,0.19,0.38,0.00,0.19,0.21,0.07,0.28,0.06,0.13,Matches,10,2,20.0,0.96,0.19,0.00,0.00,24.5,0.0,0.06,-0.2,-0.6,682.0,804.0,84.8,11931.0,3146.0,317.0,341.0,93.0,282.0,318.0,88.7,73.0,109.0,67.0,1.1,1.3,10.0,65.0,8.0,1.0,71.0,16.0,10,3.0,10.0,3.0,6.0,26.0,23.1,20.0,12.0,3.0,9.0,7,23.0,17.0,1.0
1,3333,2020-03-09,8000000,1237,GB1,James Milner,2019/2020,eng ENG,"MF,DF",Liverpool,33,1986,22,9,937,10.4,2,2,4,0,2,2,4,0,2.2,0.6,0.7,1.3,18.0,71.0,42.0,0.19,0.19,0.38,0.00,0.19,0.21,0.07,0.28,0.06,0.13,Matches,10,2,20.0,0.96,0.19,0.00,0.00,24.5,0.0,0.06,-0.2,-0.6,682.0,804.0,84.8,11931.0,3146.0,317.0,341.0,93.0,282.0,318.0,88.7,73.0,109.0,67.0,1.1,1.3,10.0,65.0,8.0,1.0,71.0,16.0,10,3.0,10.0,3.0,6.0,26.0,23.1,20.0,12.0,3.0,9.0,7,23.0,17.0,1.0
2,3333,2020-04-08,6500000,1237,GB1,James Milner,2019/2020,eng ENG,"MF,DF",Liverpool,33,1986,22,9,937,10.4,2,2,4,0,2,2,4,0,2.2,0.6,0.7,1.3,18.0,71.0,42.0,0.19,0.19,0.38,0.00,0.19,0.21,0.07,0.28,0.06,0.13,Matches,10,2,20.0,0.96,0.19,0.00,0.00,24.5,0.0,0.06,-0.2,-0.6,682.0,804.0,84.8,11931.0,3146.0,317.0,341.0,93.0,282.0,318.0,88.7,73.0,109.0,67.0,1.1,1.3,10.0,65.0,8.0,1.0,71.0,16.0,10,3.0,10.0,3.0,6.0,26.0,23.1,20.0,12.0,3.0,9.0,7,23.0,17.0,1.0
3,3333,2020-10-13,5000000,1237,GB1,James Milner,2020/2021,eng ENG,"MF,DF",Liverpool,34,1986,26,11,"1,070",11.9,0,1,1,0,0,0,3,0,0.4,0.4,0.8,1.2,21.0,100.0,51.0,0.00,0.08,0.08,0.00,0.08,0.03,0.07,0.10,0.03,0.10,Matches,12,3,25.0,1.01,0.25,0.00,0.00,24.8,0.0,0.03,-0.4,-0.4,779.0,934.0,83.4,12265.0,3618.0,435.0,466.0,93.3,270.0,315.0,85.7,56.0,92.0,60.9,1.0,0.2,13.0,66.0,21.0,3.0,100.0,26.0,19,9.0,14.0,3.0,11.0,33.0,33.3,22.0,30.0,3.0,27.0,14,40.0,16.0,1.0
4,3333,2021-03-18,3000000,1237,GB1,James Milner,2020/2021,eng ENG,"MF,DF",Liverpool,34,1986,26,11,"1,070",11.9,0,1,1,0,0,0,3,0,0.4,0.4,0.8,1.2,21.0,100.0,51.0,0.00,0.08,0.08,0.00,0.08,0.03,0.07,0.10,0.03,0.10,Matches,12,3,25.0,1.01,0.25,0.00,0.00,24.8,0.0,0.03,-0.4,-0.4,779.0,934.0,83.4,12265.0,3618.0,435.0,466.0,93.3,270.0,315.0,85.7,56.0,92.0,60.9,1.0,0.2,13.0,66.0,21.0,3.0,100.0,26.0,19,9.0,14.0,3.0,11.0,33.0,33.3,22.0,30.0,3.0,27.0,14,40.0,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6043,867688,2023-12-19,7000000,1237,GB1,Jack Hinshelwood,2023/2024,eng ENG,DF,Brighton,18-338,2005,12,8,869,9.7,3,0,3,3,0,0,1,0,1.9,1.9,0.7,2.5,16.0,19.0,38.0,0.31,0.00,0.31,0.31,0.31,0.19,0.07,0.26,0.19,0.26,Matches,14,8,57.1,1.45,0.83,0.21,0.38,9.2,0.0,0.14,1.1,1.1,4

In [77]:
#replacing the NaN values with 0 in the player stats df
total_table_df.fillna(0, inplace=True)
total_table_df

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,Name,Season,Nationality,Position,Team,Age,YOB,MatchedPlayed,Starts,MinutesPlayed,90s,Goals,Assists,GA,NonPenGoals,PensScored,PensAttempted,YellowCards,RedCards,xG,npxG,xAG,npxG + xAG,ProgressiveCarries,ProgressivePasses,ProgressivePassesReceived,G/90,A/90,GA/90,NonPenGoals/90,NonPenGA/90,xG/90,xAG/90,xA+xG /90,npxG/90,npXG + xAG /90,Matches,Shots,ShotsOnTarget,SoT%,S/90,SoT/90,G/Shot,G/SoT,AvgShotDist,FKShots,npxG/Shot,Goals - xG,npG - npxG,PassesCompleted,PassesAttempted,PercentPassesCompleted,TotalPassingDistance,ProgressedPassingDistance,ShortPassesCompleted,ShortPassesAttempted,ShortPassesPercentCompleted,MedPassesCompleted,MedPassesAttempted,MedPassesPercentCompleted,LongPassesCompleted,LongPassesAttempted,LongPassesPercentCompleted,xA,A - xAG,KeyPasses,1/3,PPen,CPen,Progressive Passes,Tackles,TacklesWon,TklsDefThird,TklsMidThird,TklsAttThird,DribblersTackled,DribblesChallenged,DrblTkl%,ChallengesLost,Blocks,BlockedShots,BlockedPasses,Interceptions,Tackles + Interceptions,Clearances,DefensiveErrors
0,3333,2019-12-10,12000000,1237,GB1,James Milner,2019/2020,eng ENG,"MF,DF",Liverpool,33,1986,22,9,937,10.4,2,2,4,0,2,2,4,0,2.2,0.6,0.7,1.3,18.0,71.0,42.0,0.19,0.19,0.38,0.00,0.19,0.21,0.07,0.28,0.06,0.13,Matches,10,2,20.0,0.96,0.19,0.00,0.00,24.5,0.0,0.06,-0.2,-0.6,682.0,804.0,84.8,11931.0,3146.0,317.0,341.0,93.0,282.0,318.0,88.7,73.0,109.0,67.0,1.1,1.3,10.0,65.0,8.0,1.0,71.0,16.0,10,3.0,10.0,3.0,6.0,26.0,23.1,20.0,12.0,3.0,9.0,7,23.0,17.0,1.0
1,3333,2020-03-09,8000000,1237,GB1,James Milner,2019/2020,eng ENG,"MF,DF",Liverpool,33,1986,22,9,937,10.4,2,2,4,0,2,2,4,0,2.2,0.6,0.7,1.3,18.0,71.0,42.0,0.19,0.19,0.38,0.00,0.19,0.21,0.07,0.28,0.06,0.13,Matches,10,2,20.0,0.96,0.19,0.00,0.00,24.5,0.0,0.06,-0.2,-0.6,682.0,804.0,84.8,11931.0,3146.0,317.0,341.0,93.0,282.0,318.0,88.7,73.0,109.0,67.0,1.1,1.3,10.0,65.0,8.0,1.0,71.0,16.0,10,3.0,10.0,3.0,6.0,26.0,23.1,20.0,12.0,3.0,9.0,7,23.0,17.0,1.0
2,3333,2020-04-08,6500000,1237,GB1,James Milner,2019/2020,eng ENG,"MF,DF",Liverpool,33,1986,22,9,937,10.4,2,2,4,0,2,2,4,0,2.2,0.6,0.7,1.3,18.0,71.0,42.0,0.19,0.19,0.38,0.00,0.19,0.21,0.07,0.28,0.06,0.13,Matches,10,2,20.0,0.96,0.19,0.00,0.00,24.5,0.0,0.06,-0.2,-0.6,682.0,804.0,84.8,11931.0,3146.0,317.0,341.0,93.0,282.0,318.0,88.7,73.0,109.0,67.0,1.1,1.3,10.0,65.0,8.0,1.0,71.0,16.0,10,3.0,10.0,3.0,6.0,26.0,23.1,20.0,12.0,3.0,9.0,7,23.0,17.0,1.0
3,3333,2020-10-13,5000000,1237,GB1,James Milner,2020/2021,eng ENG,"MF,DF",Liverpool,34,1986,26,11,"1,070",11.9,0,1,1,0,0,0,3,0,0.4,0.4,0.8,1.2,21.0,100.0,51.0,0.00,0.08,0.08,0.00,0.08,0.03,0.07,0.10,0.03,0.10,Matches,12,3,25.0,1.01,0.25,0.00,0.00,24.8,0.0,0.03,-0.4,-0.4,779.0,934.0,83.4,12265.0,3618.0,435.0,466.0,93.3,270.0,315.0,85.7,56.0,92.0,60.9,1.0,0.2,13.0,66.0,21.0,3.0,100.0,26.0,19,9.0,14.0,3.0,11.0,33.0,33.3,22.0,30.0,3.0,27.0,14,40.0,16.0,1.0
4,3333,2021-03-18,3000000,1237,GB1,James Milner,2020/2021,eng ENG,"MF,DF",Liverpool,34,1986,26,11,"1,070",11.9,0,1,1,0,0,0,3,0,0.4,0.4,0.8,1.2,21.0,100.0,51.0,0.00,0.08,0.08,0.00,0.08,0.03,0.07,0.10,0.03,0.10,Matches,12,3,25.0,1.01,0.25,0.00,0.00,24.8,0.0,0.03,-0.4,-0.4,779.0,934.0,83.4,12265.0,3618.0,435.0,466.0,93.3,270.0,315.0,85.7,56.0,92.0,60.9,1.0,0.2,13.0,66.0,21.0,3.0,100.0,26.0,19,9.0,14.0,3.0,11.0,33.0,33.3,22.0,30.0,3.0,27.0,14,40.0,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6043,867688,2023-12-19,7000000,1237,GB1,Jack Hinshelwood,2023/2024,eng ENG,DF,Brighton,18-338,2005,12,8,869,9.7,3,0,3,3,0,0,1,0,1.9,1.9,0.7,2.5,16.0,19.0,38.0,0.31,0.00,0.31,0.31,0.31,0.19,0.07,0.26,0.19,0.26,Matches,14,8,57.1,1.45,0.83,0.21,0.38,9.2,0.0,0.14,1.1,1.1,4

In [78]:
path = "/content/drive/MyDrive/Senior year/Capstone/data/GK Master Table.csv"
gkstats = pd.read_csv(path, encoding='iso-8859-1')
gkstats

,Name,Nationality,Pos,Team,Age,YOB,MatchedPlayed,Starts,MinutesPlayed,90s,GoalsAgainst,GoalsAgainst/90,SotAgainst,Saves,SavePercentage,Wins,Draws,Losses,CleanSheets,CleanSheetPercent,PensAttempted,PensAllowed,PensSaved,PensMissed,PenSavePercent,Matches,Season
0,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,90s,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKatt,PKA,PKsv,PKm,Save%,Matches,2023/2024
1,Alisson,br BRA,GK,Liverpool,31-164,1992,21,21,"1,890",21.0,19,0.90,73,55,75.3,13,6,2,7,33.3,1,1,0,0,0.0,Matches,2023/2024
2,Alphonse Areola,fr FRA,GK,West Ham,31-016,1993,25,25,"2,204",24.5,40,1.63,144,104,75.0,10,7,8,4,16.0,6,4,2,0,33.3,Matches,2023/2024
3,Daniel Bentley,eng ENG,GK,Wolves,30-245,1993,4,2,293,3.3,4,1.23,12,8,66.7,1,0,1,1,50.0,0,0,0,0,NaN,Matches,2023/2024
4,Martin Dúbravka,sk SVK,GK,Newcastle Utd,35-059,1989,14,13,"1,175",13.1,30,2.30,89,58,69.7,4,2,7,3,23.1,4,3,1,0,25.0,Matches,2023/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Roberto,es ESP,GK,West Ham,33,1986,8,7,687,7.6,17,2.23,51,35,68.6,0,1,6,0,0.0,1,1,0,0,0.0,Matches,2019/2020
203,Mathew Ryan,au AUS,GK,Brighton,27,1992,38,38,"3,420",38.0,54,1.42,168,116,69.0,9,14,15,9,23.7,2,2,0,0,0.0,Matches,2019/2020
204,Kasper Schmeichel,dk DEN,GK,Leicester City,32,1986,38,38,"3,420",38.0,41,1.08,135,94,75.6,18,8,12,13,34.2,10,8,2,0,20.0,Matches,2019/2020
205,Jed Steer,eng ENG,GK,Aston Villa,26,1992,1,1,7,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,2019/2020


In [79]:
#replacing the NaN values with 0 in the gk stats df
gkstats.fillna(0, inplace=True)
gkstats

,Name,Nationality,Pos,Team,Age,YOB,MatchedPlayed,Starts,MinutesPlayed,90s,GoalsAgainst,GoalsAgainst/90,SotAgainst,Saves,SavePercentage,Wins,Draws,Losses,CleanSheets,CleanSheetPercent,PensAttempted,PensAllowed,PensSaved,PensMissed,PenSavePercent,Matches,Season
0,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,90s,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKatt,PKA,PKsv,PKm,Save%,Matches,2023/2024
1,Alisson,br BRA,GK,Liverpool,31-164,1992,21,21,"1,890",21.0,19,0.90,73,55,75.3,13,6,2,7,33.3,1,1,0,0,0.0,Matches,2023/2024
2,Alphonse Areola,fr FRA,GK,West Ham,31-016,1993,25,25,"2,204",24.5,40,1.63,144,104,75.0,10,7,8,4,16.0,6,4,2,0,33.3,Matches,2023/2024
3,Daniel Bentley,eng ENG,GK,Wolves,30-245,1993,4,2,293,3.3,4,1.23,12,8,66.7,1,0,1,1,50.0,0,0,0,0,0,Matches,2023/2024
4,Martin Dúbravka,sk SVK,GK,Newcastle Utd,35-059,1989,14,13,"1,175",13.1,30,2.30,89,58,69.7,4,2,7,3,23.1,4,3,1,0,25.0,Matches,2023/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Roberto,es ESP,GK,West Ham,33,1986,8,7,687,7.6,17,2.23,51,35,68.6,0,1,6,0,0.0,1,1,0,0,0.0,Matches,2019/2020
203,Mathew Ryan,au AUS,GK,Brighton,27,1992,38,38,"3,420",38.0,54,1.42,168,116,69.0,9,14,15,9,23.7,2,2,0,0,0.0,Matches,2019/2020
204,Kasper Schmeichel,dk DEN,GK,Leicester City,32,1986,38,38,"3,420",38.0,41,1.08,135,94,75.6,18,8,12,13,34.2,10,8,2,0,20.0,Matches,2019/2020
205,Jed Steer,eng ENG,GK,Aston Villa,26,1992,1,1,7,0.1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Matches,2019/2020


In [80]:
keeper_df = pd.merge(merged_table, gkstats, on=['Name', 'Season'], how='inner')
keeper_df

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,Name,Season,Nationality,Pos,Team,Age,YOB,MatchedPlayed,Starts,MinutesPlayed,90s,GoalsAgainst,GoalsAgainst/90,SotAgainst,Saves,SavePercentage,Wins,Draws,Losses,CleanSheets,CleanSheetPercent,PensAttempted,PensAllowed,PensSaved,PensMissed,PenSavePercent,Matches
0,13572,2019-12-10,1500000,1010,GB1,Ben Foster,2019/2020,eng ENG,GK,Watford,36,1983,38,38,"3,420",38.0,64,1.68,178,116,68.5,8,10,20,9,23.7,9,8,1,0,11.1,Matches
1,13572,2020-04-08,1200000,1010,GB1,Ben Foster,2019/2020,eng ENG,GK,Watford,36,1983,38,38,"3,420",38.0,64,1.68,178,116,68.5,8,10,20,9,23.7,9,8,1,0,11.1,Matches
2,13572,2021-06-30,300000,1010,GB1,Ben Foster,2021/2022,eng ENG,GK,Watford,38,1983,26,26,"2,340",26.0,49,1.88,116,67,60.3,5,5,16,4,15.4,4,3,1,0,25.0,Matches
3,13572,2021-12-23,300000,1010,GB1,Ben Foster,2021/2022,eng ENG,GK,Watford,38,1983,26,26,"2,340",26.0,49,1.88,116,67,60.3,5,5,16,4,15.4,4,3,1,0,25.0,Matches
4,14555,2020-10-13,800000,281,GB1,Scott Carson,2020/2021,eng ENG,GK,Manchester City,34,1985,1,1,90,1.0,3,3.00,5,1,60.0,1,0,0,0,0.0,2,1,1,0,50.0,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,585550,2022-09-15,15000000,180,GB1,Gavin Bazunu,2022/2023,ie IRL,GK,Southampton,20,2002,32,32,"2,880",32.0,56,1.75,120,67,54.2,6,6,20,4,12.5,2,1,1,0,50.0,Matches
480,585550,2022-11-03,15000000,180,GB1,Gavin Bazunu,2022/2023,ie IRL,GK,Southampton,20,2002,32,32,"2,880",32.0,56,1.75,120,67,54.2,6,6,20,4,12.5,2,1,1,0,50.0,Matches
481,585550,2023-03-16,18000000,180,GB1,Gavin Bazunu,2022/2023,ie IRL,GK,Southampton,20,2002,32,32,"2,880",32.0,56,1.75,120,67,54.2,6,6,20,4,12.5,2,1,1,0,50.0,Matches
482,566799,2023-07-12,15000000,1132,GB1,James Trafford,2023/2024,eng ENG,GK,Burnley,21-156,2002,28,28,"2,520",28.0,62,2.21,166,106,65.7,3,5,20,2,7.1,5,5,0,0,0.0,Matches


Training/Testing Split

In [81]:
total_table_two_df = total_table_df

In [82]:
total_table_df.drop(columns = ['current_club_id'], inplace = True)

In [83]:
total_table_df.drop(columns = ['Matches'], inplace = True)

In [84]:
total_table_df.drop(columns = ['Name'], inplace = True)

In [85]:
total_table_df.drop(columns = ['date'], inplace = True)

In [86]:
total_table_df.drop(columns = ['player_club_domestic_competition_id'], inplace = True)

In [87]:
# Splitting the age column on '-' and extracting the first part
total_table_df['Age'] = total_table_df['Age'].str.split('-').str[0]

# Converting the age column to integer type
total_table_df['Age'] = total_table_df['Age'].astype(int)

In [88]:
#Removes commas for minutes played
def convert_to_numeric(value):
    if isinstance(value, str):
        return int(value.replace(",", ""))  # Assuming values are integers
    else:
        return value

total_table_df['MinutesPlayed'] = total_table_df['MinutesPlayed'].apply(convert_to_numeric)

In [89]:
column_names = total_table_df.columns.tolist()
print(column_names)

['player_id', 'market_value_in_eur', 'Season', 'Nationality', 'Position', 'Team', 'Age', 'YOB', 'MatchedPlayed', 'Starts', 'MinutesPlayed', '90s', 'Goals', 'Assists', 'GA', 'NonPenGoals', 'PensScored', 'PensAttempted', 'YellowCards', 'RedCards', 'xG', 'npxG', 'xAG', 'npxG + xAG', 'ProgressiveCarries', 'ProgressivePasses', 'ProgressivePassesReceived', 'G/90', 'A/90', 'GA/90', 'NonPenGoals/90', 'NonPenGA/90', 'xG/90', 'xAG/90', 'xA+xG /90', 'npxG/90', 'npXG + xAG /90', 'Shots', 'ShotsOnTarget', 'SoT%', 'S/90', 'SoT/90', 'G/Shot', 'G/SoT', 'AvgShotDist', 'FKShots', 'npxG/Shot', 'Goals - xG', 'npG - npxG', 'PassesCompleted', 'PassesAttempted', 'PercentPassesCompleted', 'TotalPassingDistance', 'ProgressedPassingDistance', 'ShortPassesCompleted', 'ShortPassesAttempted', 'ShortPassesPercentCompleted', 'MedPassesCompleted', 'MedPassesAttempted', 'MedPassesPercentCompleted', 'LongPassesCompleted', 'LongPassesAttempted', 'LongPassesPercentCompleted', 'xA', 'A - xAG', 'KeyPasses', '1/3', 'PPen', 

In [90]:
chosen_column = 'Nationality'

# Create a dictionary to map unique values to unique identifiers
unique_values = total_table_df[chosen_column].unique()
id_mapping = {value: idx for idx, value in enumerate(unique_values, start=1)}

# Create a new column in the DataFrame to store the primary IDs
total_table_df['nationality_id'] = total_table_df[chosen_column].map(id_mapping)

# Adding the id to the copy df so we can merge them when its time to make the front end
total_table_two_df['nationality_id'] = total_table_df['nationality_id']

In [91]:
# Now repeating the process for our other features
# Team
chosen_column = 'Team'

# Create a dictionary to map unique values to unique identifiers
unique_values = total_table_df[chosen_column].unique()
id_mapping = {value: idx for idx, value in enumerate(unique_values, start=1)}

# Create a new column in the DataFrame to store the primary IDs
total_table_df['team_id'] = total_table_df[chosen_column].map(id_mapping)

# Adding the id to the copy df so we can merge them when its time to make the front end
total_table_two_df['team_id'] = total_table_df['team_id']

In [92]:
# Repeating for position
chosen_column = 'Position'

# Create a dictionary to map unique values to unique identifiers
unique_values = total_table_df[chosen_column].unique()
id_mapping = {value: idx for idx, value in enumerate(unique_values, start=1)}

# Create a new column in the DataFrame to store the primary IDs
total_table_df['position_id'] = total_table_df[chosen_column].map(id_mapping)

# Adding the id to the copy df so we can merge them when its time to make the front end
total_table_two_df['position_id'] = total_table_df['position_id']

In [93]:
# Repeating for season
chosen_column = 'Season'

# Create a dictionary to map unique values to unique identifiers
unique_values = total_table_df[chosen_column].unique()
id_mapping = {value: idx for idx, value in enumerate(unique_values, start=1)}

# Create a new column in the DataFrame to store the primary IDs
total_table_df['season_id'] = total_table_df[chosen_column].map(id_mapping)

# Adding the id to the copy df so we can merge them when its time to make the front end
total_table_two_df['season_id'] = total_table_df['season_id']

In [94]:
#Removing all of the categorical features now that we have the ID
total_table_df.drop(columns = ['Season'], inplace = True)
total_table_df.drop(columns = ['Position'], inplace = True)
total_table_df.drop(columns = ['Team'], inplace = True)
total_table_df.drop(columns = ['Nationality'], inplace = True)

In [95]:
X = total_table_df.drop(['market_value_in_eur'], axis =1)
y = total_table_df['market_value_in_eur']


In [96]:
# split into training and temp
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# split into testing and validation
X_test, X_valid, y_test, y_valid = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [97]:
print("Training set shapes:", X_train.shape, y_train.shape)
print("Testing set shapes:", X_test.shape, y_test.shape)
print("Validation set shapes:", X_valid.shape, y_valid.shape)

Training set shapes: (4233, 85) (4233,)
Testing set shapes: (907, 85) (907,)
Validation set shapes: (908, 85) (908,)


In [98]:
# Standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

In [103]:
X_train

array([[-0.13280792, -0.15487566,  0.01677771, ...,  1.70442298,
        -0.91846635,  0.10029926],
       [-1.15422016,  1.05158124, -0.89373576, ...,  0.87425835,
         0.81005118,  0.80555967],
       [-0.40715687,  0.3277071 , -0.6661074 , ..., -1.61623556,
        -0.48633697, -1.31022156],
       ...,
       [ 2.12160444, -1.60262395,  1.38254792, ...,  1.01261912,
        -0.48633697, -0.60496115],
       [ 1.19271641, -0.87874981,  1.15491956, ...,  0.45917603,
         0.81005118,  0.80555967],
       [-0.99123706,  1.05158124, -1.12136413, ..., -0.09426706,
        -0.91846635, -0.60496115]])

In [99]:
# create the lasso model
lasso_cv = LassoCV(alphas=[0.001, 0.01, 0.1, 1, 10], cv=5, random_state=42)

lasso_cv.fit(X_train, y_train)

print('The optimal alpha is', lasso_cv.alpha_)

# finding the predicted probability
y_pred_proba = lasso_cv.predict(X_test)
# converting to binary predictions
y_pred = (y_pred_proba > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

The optimal alpha is 10.0
Accuracy: 0.0
